# Introduction

In this notebook we will need two of the Maxwell equations in vacuum (in SI units):
\begin{align}
\nabla\times\vec{B}-\frac{1}{c^2}\frac{\partial\vec{E}}{\partial t} &= \mu_0\vec{j} \tag{Ampere's law}\\
\nabla\cdot\vec{B} &= 0  \tag{Gauss's law for magnetism}
\end{align}
and, off course, the force equation,
$$ \vec{F} = q\left(\vec{E}+\vec{v}\times\vec{B}\right) \tag{Lorentz force}$$

it is also convenient to recall how the divergence of a vector is written in cylindrical coordinates:
$$ \nabla\cdot\vec{A} = \frac{1}{r}\frac{\partial}{\partial r}(rA_r)+\frac{\partial A_\theta}{r\partial\theta}+\frac{\partial A_z}{\partial z} $$
We will use also the Gauss theorem,
$$ \int_V \nabla\cdot\vec{A}\;dV = \oint_S \vec{A}\cdot\vec{n}\;dS, $$
and also the concept of magnetic moment, $\mu$, which, for a closed loop of area $A$ and current $I$, has the value $\mu = IA$.

The subject of this notebook is covered in the bibliography in the following chapters:

* Chen<cite data-cite="chen1974"></cite>: chapter Two, section 2.3
* Nicholson<cite data-cite="nicholson1983"></cite>: chapter 2, section 2.3, 2.4 and 2.6
* Bittencourt<cite data-cite="Bittencourt2004"></cite>: chapter 3
* Goldston<cite data-cite="Goldston1995"></cite>: chapter 3

The examples are prepared with the help of two scientific software packages, *Scipy*<cite data-cite="jones2001"></cite> and *IPython*<cite data-cite="PER-GRA:2007"></cite>.

# Type of nonuniformities

We start by studying four types of space changes in $\vec{B}$:
<table style="width:90%">
<caption>Types of nonuniform magnetic field</caption>
  <tr>
    <td><img src="images/gradB.png" style="width:200px;height:162px"/>Grad B</td>
    <td><img src="images/curvB.png" style="width:200px;height:162px"/>Curvature</td>
    <td><img src="images/curvgradB.png" style="width:200px;height:162px"/>Grad B and curvature</td>
    <td><img src="images/divB.png" style="width:200px;height:162px"/>Divergence B</td>
  </tr>
  <tr>
  <td>1) $\partial B_z/\partial x,\,\partial B_z/\partial y$</td>
  <td>2) $\partial B_x/\partial z,\,\partial B_y/\partial z$</td>
  <td>1)+2)</td>
  <td>3) $\partial B_x/\partial x,\,\partial B_y/\partial y,\,\partial B_z/\partial z$</td>
  </tr>
</table>

# $\nabla{B} \perp \vec{B}$: Grad B drift

**Important**: $\color{blue}{Assumption:} r_L \ll B/|\nabla B|$

The magnetic field lines are straight but their density changes in the plane $\perp$ to $\vec{B}$.
$$ \vec{B} = \vec{B}_0 + (\vec{r}\cdot\nabla)\vec{B} + \cdots $$

To simplify, we consider that $\vec{B}$ varies only with *y*:
$$ \vec{B} = B_{gc,0}\vec{u}_z + (y-y_{gc})\frac{d B}{d y}\vec{u}_z $$
*Note*: according to Maxwell equations, as $\nabla\times\vec{B} \ne \vec{0}$, to have this field we need distributed volume currents...

## What happens? How is the movement?

$\color{red}{Discussion...}$

We start from the Lorentz equation without electric field, $\vec{F}=q\vec{v}\times\vec{B}$. 

If we average on the gyroperiods, the $x$ average component is zero, $\overline{F_x}=0$. For the y component, we have $F_y= - q v_x B_z(y)$. using the results of the previous notebook for $v_x$ and $y-y_0$:
$$ F_y = -q v_\perp \cos(\omega_c t)\left[B_{gc,0}\pm r_L\cos(\omega_c t)\frac{\partial B}{\partial y}\right] $$
Averaging in a gyroperiod we obtain
$$ \overline{F_y} = \mp\frac{1}{2}qv_\perp r_L\frac{\partial B}{\partial y} $$
(*Note*: $\overline{\cos^2(\omega_ct)} = 1/2$).

This force is responsible for a drift of the guiding center. Using the result from the previous notebook for the value of this drift we write
$$ \vec{v}_{grad} = \mp \frac{1}{2}\frac{v_\perp r_L}{B}\frac{\partial B}{\partial y}\vec{u}_x $$
Finally, generalizing for a gradient $\nabla{B} \perp \vec{B}$, we reach at the result
$$ v_{\nabla B} = \pm \frac{1}{2}v_\perp r_L\frac{\vec{B}\times\nabla{B}}{B^2} $$

---

## Practice:

Let's see how important is the assumption $r_L \ll B/|\nabla B|$. We can compute the trajectories for an arbitrary value of ${\partial B}/{\partial y}$ and see what happens... We start by importing some libraries:

In [ ]:
%matplotlib 
#inline
import numpy as np
from ipywidgets import interact
from scripts.trajectories import *
plt.style.use('ggplot')

And we can compute the trajectory for a field with a perpendicular gradient:

In [ ]:
def gradB(Q, t, qbym, E0, B0, keywords):
    """Equations of movement for a grad-B magnetic field.
    
    Positional arguments:
    Q -- 6-dimension array with (x,y,z) values of position and velocity on t-dt
    t -- next time (not used here but passed by odeint)
    qbym -- q/m
    E0, B0 -- arrays with electric and magnetic field values
    Keyword arguments:
    dBdx, dBdy -- The gradient values in x and y, respectively.
    
    Return value:
    Array with dr/dt and dv/dt values.
    
    Note: The condition rL/(grad(B)/B) == coeficients of Q[:1] << 1 should be obeyed.
    """

    gradx, grady = "grad" in keywords.keys() and keywords["grad"] or [0, 0]
    x, y = Q[:2]
    B = B0*np.array([1,1,1+x*gradx+y*grady])
    
    v = Q[3:]                                 # Velocity
    dvdt = qbym*np.cross(v,B)                 # Acceleration
    return np.concatenate((v,dvdt))

def gradBdrift(gradx=0.2, grady=0.7):
    """Movement under a grad B perpendicular to B"""
    re, rp = computeTrajectories(gradB, grad=[gradx/10,grady/10]) # NOTE the /10 !
    plotGradB(re,rp,gradx,grady)

dummy = interact(gradBdrift, gradx=(-2.5,2.5), grady=(-2.5,2.5))

* Electrons and positive ions drift in oposite directions $\Rightarrow$ **net current**!

# Curvature drift

$\color{blue}{Assumption:}$ B field lines locally curved (but constant) with radius of curvature $\vec{R}_c$.

Achieved also with volume currents.

Centrifugal force in the radial direction:
$$ \vec{F}_{cf} = \frac{mv_\parallel^2}{R_c}\vec{u}_r = mv_\parallel^2\frac{\vec{R}_c}{R_c^2} $$
and using the equation above for a generic force:
$$ \vec{v}_{curv} = \frac{mv_\parallel^2}{q B^2}\frac{\vec{R}_c\times\vec{B}}{R_c^2}$$
In 'vacuum fields' (without volume currents) B must fall in the perpendicular direction as $|B|\propto 1/R_c$ and $\nabla|B|/B= -\vec{R}_c/R_c^2$. Thus,

$$ \vec{v}_{curv} = \pm \frac{v_\parallel^2}{\omega_c}\frac{\vec{B}\times\nabla B}{B^2} $$

# Total drift

* The curvature and grad drift add;
* In opposite directions for charges of opposite signs;
* Proportional to the particle energy.
$$ \vec{v}_d = \left( \frac{1}{2}mv_\perp^2 + mv_\parallel^2 \right)\frac{1}{qB^2}\frac{\vec{R}_c\times\vec{B}}{R_c^2}$$
* For a Maxwellian isotropic distribution both terms give the same contribution.

# Gradient along $B$

Let us analyse the case where the magnetic field increases along his direction, i.e. for $\vec{B}=B_z\vec{u}_z$, we have $\partial B_z / \partial z > 0$.

As we must have $\nabla\cdot\vec{B}=0$, (assuming, to simplify, that $B_\theta = 0$) then $\frac{1}{r}\frac{\partial}{\partial r}(rB_r)< 0$.

Assuming again that $r_L \ll B/|\nabla B|$, we can compute an *average* value for $\overline{B_r}$. Taking a small cylindrical volume centered along a magnetic field line, integrating and using the Gauss theorem, we can write $\pi(\delta r)^2\delta l(d B/d z)+2\pi\delta r\delta l\overline{B_r}=0$ and

$$ \overline{B_r} = - \frac{\delta r}{2}\frac{d B}{d z}. $$

Now, if $\delta r$ is the Larmor radius, it is this field that intervenes in the Lorentz force. Taking the time average over the gyro-period, we obtain
$$ \overline{F_\parallel} = - \frac{|q|v_\perp^2}{2\omega_c}\frac{d B}{d z} = -\frac{W_\perp}{B}\frac{d B}{d z}. $$

* Force in the direction opposite to the field gradient for **both** positive and negative charges.

## Magnetic moment

In the previous notebook we have introduced the magnetic moment for the gyrating particle: 
$$ \mu = \frac{|q|v_\perp^2}{2\omega_c} = \frac{m v_\perp^2}{2 B} = \frac{W_\perp}{B}. $$

Now, it is time to see the importance of $\mu$. Let's look at the conservation laws for this case:

* Angular momentum: As the force $\overline{F_\parallel}$ is constant, the angular momemtum, $\vec{L} = m\vec{r}_L\times\vec{v}$ is constant. Note that $\vec{L} = m r_L v_\perp\vec{u}_\parallel$;
* Kinetic energy: In the presence of a static magnetic field, the total kinetic energy must be conserved.

Both laws imply that the $\mu$ is a constant of motion as we will show:

* Conservation of the angular momentum: $\vec{L} = m r_L v_\perp\vec{u}_\parallel \equiv (2m/|q|)\vec{\mu}\,$ as $\,d\vec{L}/dt = 0 \Rightarrow \vec{\mu}$ is constant;
* Conservation of the kinetic energy: We start by using the above result we write
$$ m\frac{dv_\parallel}{dt} = -\mu\frac{dB}{ds},$$
where we have parameterized the distance along the field line, $s$. Multiplying both sides of this equation by $v_\parallel=ds/dt$ we obtain
$$ \frac{d}{dt}\left(\frac{m v_\parallel^2}{2}\right) = -\mu\frac{dB}{dt}.$$

For the total kinetic energy, we write
$$ \frac{d}{dt}\left(\frac{mv_\parallel^2}{2}+\frac{mv_\perp^2}{2}\right) = \frac{d}{dt}\left(\frac{mv_\parallel^2}{2}+\mu B\right) = 0 $$
Using the above result we have
$$ -\mu\frac{dB}{dt}+\frac{d}{dt}(\mu B) = 0 \Rightarrow \frac{d\mu}{dt} = 0. $$

* $\mu$ is an invariant!
* $v_\perp$ has to increase when the particle moves to higher $B$ regions;
* $v_\parallel$ decreases as $v_\perp$ increases.

## Magnetic mirrors

This effect is used to confine plasmas. Let's supose that we have the a magnetic field with the following configuration of field lines:
<img src="images/MagnMirror.png" style="width:500px;height:456px"/>
The parallel velocity of a particle with energy $W$ and magnetic moment $\mu$ has to obey the equation
$$ \frac{mv_\parallel^2}{2} = W - \mu B(z) $$
and will have a reflection point at $z_{max}:\,B(z_{max}) = W/\mu$.

**Are all particles trapped?**

Let as consider a particle moving around a field line with and minimum value, $B_{min}$, in the midplane, $m$, and $B_{max}$ at the mirror throat. The limiting conditions to trap particles are (using the result above):
\begin{align}
\left.W_\perp\right|_m &= \mu B_{min} = W B_{min}/B_{max} \\
\left.W_\parallel\right|_m &= W(1 - B_{min}/B_{max})
\end{align}
Particles with higher value of $\left.W_\parallel\right|_m/W$ can escape the trap! This condition defines a 'loss cone'.

* Exercise: Show that the equation for the 'loss cone' is 
$$\frac{|v_\parallel|}{|v_\perp|} = \left(\frac{B_{max}}{B_{min}}-1\right)^{1/2} $$
and this defines a critical angle.

---

## Practice:

**Note:** The following code is still under development!

In [12]:
%matplotlib qt4

from mpl_toolkits.mplot3d import Axes3D

def Motion(Q,t,qbym,gamma):
    """Equations of movement"""
    global B0, mu, zMax
    
    r = Q[:3]; v = Q[3:]
    drdt = v                                   # Velocity
    vperp = np.sqrt(v[0]**2+v[1]**2)           # Perpendicular|B velocity
    if np.abs(r[2]) <= zMax:
        Bz = B0*(1+gamma*r[2]**2)
        omega_c = np.abs(qbym)*Bz              # Cyclotron frequency
        rLe = vperp/omega_c #; rLp = Mp/me*rLe # Larmor radius
        Br = -rLe*B0*gamma*np.abs(r[2])
        theta = omega_c*t
        B = np.array([Br*np.cos(theta),Br*np.sin(theta),Bz])
        dvdt = qbym*np.cross(v,B)              # Acceleration
        dvdt[2] = - mu*2*B0*gamma*r[2]
    else:
        dvdt = np.zeros(3)
    return np.concatenate((drdt,dvdt))

def mirrorB(Bratio=1.2, vratio=0.6, gamma=1e-4):
    """Movement with a grad B parallel to B"""
    global B0, mu, zMax
    q = 1; me = 1; Mp = 10*me; B0 = 1; vpar0 = 1
    # Initial values
    vperp0 = vpar0/vratio
    rL0 = vperp0/(q/me*B0)                     # Larmor radius
    x0 = rL0; y0 = 0; vx0 = 0; vy0 = vperp0
    r0 = np.array([x0,y0,0]); v0 = np.array([vx0,vy0,vpar0])  # Position and velocity
    Wperp0 = me*vperp0**2/2                    # Perp. kinetic energy
    mu = Wperp0/B0                             # Magnetic moment
    W = Wperp0 + me*vpar0**2/2                 # Kinetic energy
    BMax = Bratio*B0
    BMax1 = W/mu
    zMax = np.sqrt((BMax/B0-1)/gamma)
    
    tmax = 300; t = np.linspace(0,tmax,10*tmax)
  
    # Integrates the equations of movement
    Q0 = np.concatenate((r0,v0))
    Qe = odeint(Motion, Q0, t, args=(-q/me,gamma))  # electrons

    # Plot the trajectories and Larmor radius
    fig = plt.figure(figsize=(8,6))
    fig.clf()
    ax = fig.gca(projection='3d')
    ax.scatter(r0[0],r0[1],r0[2],c='red')      # Starting point
    # Legibility
    ax.set_title("Trajectories",fontsize=18)
    ax.set_xlabel("X Axis",fontsize=16); ax.set_ylabel("Y Axis",fontsize=16)
    ax.set_zlabel("Z Axis",fontsize=16)
    ax.text(17,15,200, "$\\uparrow\\, \\vec{B}$", color="red",fontsize=20)
    Q = Qe[np.where(Qe[:,2]<1.02*zMax)]
    ax.plot(Q[:,0],Q[:,1],Q[:,2])              # Electron trajectory
    # Final points
    ax.scatter(Q[-1,0],Q[-1,1],Q[-1,2],c='green', marker='>')

    print('Magnetic moment   Kinetic energy  rB_conf  rB_max  \
zM_conf\n   mu = {:.3f}       W = {:.3f}      {:.3f}    {:.3f} \
 {:.3f}'.format(mu,W,BMax,BMax1,zMax))
    
mirrorB()

Magnetic moment   Kinetic energy  rB_conf  rB_max  zM_conf
   mu = 1.389       W = 1.889      1.200    1.360  44.721


## Applications

### In plasma machines

<table>
<tr>
<td><img src="images/MagMirMach1.png" style="width:250px;height:228px"/></td>
<td><img src="images/MagMirMach2.png" style="width:250px;height:228px"/></td>
</tr>
</table>

### In Nature

<img src="images/MagMirNature4.png" style="width:450px;height:256px"/>
1 - Cyclotronic motion; 2 - Mirror effect; 3 - Curvature drift.

<table>
<tr>
<td><img src="images/MagMirNature1.png" style="width:250px;height:228px"/></td>
<td><img src="images/MagMirNature2.png" style="width:250px;height:228px"/></td>
</tr>
</table>

New effect discovered in September 2012! - the Van Allen belts:

<img src="images/MagMirNature3.png" style="width:500px;height:300px"/>

# Nonuniform $\vec{E}$ field

To finish this notebook we still have to look at the case of a nonuniform electric field. So simplify let us assume a sinusoidal variation in the $x$ direction:
$$ \vec{E} = E_0 \cos(kx)\vec{u}_x, $$
and we take $k$ such that the wavelength $\lambda=2\pi/k$ is large comparing with $r_L$.

From the Lorentz force equation we have
\begin{align}
\dot{v}_x &= \frac{q B}{m}v_y + \frac{q}{m}E_x(x)  \\
\dot{v}_y &= -\frac{q B}{m}v_x \\
\end{align}
and, we focus our attention in the equation for $\ddot{v}_y$:
$$ \ddot{v}_y = -\omega_c^2 v_y - \omega_c^2 \frac{E_x(x)}{B}. $$
To solve this we need to know the field at $x$ but this depends on the orbit equation that we are trying to obtain... That's when our approximation is needed: We use the *undisturbed orbit*, $x = x_0 + r_L\sin \omega_c t$.

As before, we expect to have a drift, $v_E$, superimposed to a gyration movement. As we have done before, to find $v_E$ we average on the gyroperiods to obtain $\overline{\ddot{v}}_{x,y} = 0$. As this implies $\overline{v}_x = 0$ (*Question*: why?) we only have a drift along $y$. 

To reach to a value for $v_E$ we need to average the $\overline{\cos(x_0 + r_L\sin \omega_c t)}$ term. After expanding the cosine, we need again our assumption, $k r_L \ll 1$ to use a Taylor expansion. After some mathematics we obtain
$$ \overline{v}_y = - \frac{E_x(x_0)}{B}\left(1-\frac{1}{4}k^2r_L^2\right), $$
where we have put $E_0\cos(kx_0)=E_x(x_0)$.

Or, in vector form,
$$ \vec{v}_E = \frac{\vec{E}\times\vec{B}}{B^2}\left(1-\frac{1}{4}k^2r_L^2\right). $$
This result introduces a correction frm the inhomogeneity on our previous result for the $\vec{E}\times\vec{B}$ drift. 

For an arbitrary variation of $\vec{E}$, this term has the value
$$ \vec{v}_E = \left(1+\frac{1}{4}r_L^2\nabla^2\right)\frac{\vec{E}\times\vec{B}}{B^2}. $$


**Electrons and ions have different Larmor radius** $\Rightarrow$ charge separation $\Rightarrow$ possibility of plasma instabilities (*drift instability*).